<a href="https://colab.research.google.com/github/Nomi230/ColabWork/blob/main/decode_without_lm_working_updated_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] ="/scratch/project_2007260/HF_cache"
os.environ['HF_DATASETS_CACHE'] ="/scratch/project_2007260/HF_cache"
os.environ['PYTHONPATH']="/scratch/project_2007260/miniconda/envs/pytest"
os.environ['CONDA_PREFIX']="/scratch/project_2007260/miniconda/envs/pytest"


In [ ]:
from datasets import load_dataset
import IPython.display as ipd
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, WhisperProcessor, WhisperForConditionalGeneration, AutoProcessor,HubertModel, HubertForSequenceClassification, HubertForCTC
from transformers import Wav2Vec2ProcessorWithLM
import torch
import pyctcdecode
import soundfile as sf
import torch
from jiwer import wer
import torchaudio
import torchaudio.transforms as T
from pyctcdecode import build_ctcdecoder
import soundfile as sf
import pandas
import librosa
from jiwer import wer, process_words, visualize_alignment, Compose, ToLowerCase, RemoveWhiteSpace, RemoveMultipleSpaces, ReduceToListOfListOfWords

/users/abbasino/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-07-27 18:59:46.808171: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-27 18:59:47.205765: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-27 18:59:50.755799: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-

In [ ]:
pytorch_device = torch.device("cuda:0")
torch.set_num_threads(1)
sample_rate=16000

In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-base.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base.en").to("cuda:0")

wav_path="/scratch/project_2007260/cv_aged_voice/FinalDataset/ASR/wavp_eighties_combined_test"
text_path="/scratch/project_2007260/cv_aged_voice/FinalDataset/ASR/tid_eighties_combined_test"

wav_l = [temp.strip() for temp in open(wav_path).readlines()]
text_l = [temp.strip() for temp in open(text_path).readlines()]
df = pandas.DataFrame(data={"file": wav_l, "audio": wav_l, "text": text_l})

df.to_csv("test.csv", sep=",",index=False)

dataFiles = {'test': "test.csv"}
ld = load_dataset('csv', data_files=dataFiles)

eval_data = ld["test"]
sample_rate=16000
device="cuda:0"

ref = []
res = []
punc = '''!()-[]{};.:'"\,<>./?@#$%^&*_~'''

for i in range(len(eval_data)):
    audio_sample = eval_data[i]
    if os.path.exists(audio_sample["file"]):
     audio, _ = librosa.load(audio_sample["file"], sr=16000)
     test_str=audio_sample["text"].lower()
     stk=""
     for ele in test_str:
        if ele not in punc:
            stk+=ele
     ref.append(stk)

     input_features = processor(audio, sampling_rate=sample_rate, return_tensors="pt").input_features.to(device)
     predicted_ids = model.generate(input_features)
     transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

     test_str=transcription[0].lower()
     stk=""
     for ele in test_str:
        if ele not in punc:
            stk+=ele
     res.append(stk)
     if i%1000 == 0:
        print(ref[i],res[i])


transformation = Compose([
    ToLowerCase(),
    RemoveWhiteSpace(replace_by_space=True),
    RemoveMultipleSpaces(),
    ReduceToListOfListOfWords(word_delimiter=" ")
])

out = process_words(ref,res)

print("WER Whisper:", wer(ref,res,truth_transform=transformation,hypothesis_transform=transformation)*100, "Substitutions:", out.substitutions, " Deletions:", out.deletions, " Insertions:",out.insertions)


SyntaxError: invalid syntax (3835713575.py, line 17)

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self").to("cuda:0")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

wav_path="/scratch/project_2007260/cv_aged_voice/FinalDataset/ASR/wavp_eighties_combined_test"
text_path="/scratch/project_2007260/cv_aged_voice/FinalDataset/ASR/tid_eighties_combined_test"

wav_l = [temp.strip() for temp in open(wav_path).readlines()]
text_l = [temp.strip() for temp in open(text_path).readlines()]
df = pandas.DataFrame(data={"file": wav_l, "audio": wav_l, "text": text_l})

df.to_csv("test.csv", sep=",",index=False)

dataFiles = {'test': "test.csv"}
ld = load_dataset('csv', data_files=dataFiles)

eval_data = ld["test"]


sample_rate=16000
device="cuda:0"

ref = []
res = []
punc = '''!()-[]{};.:'"\,<>./?@#$%^&*_~'''

for i in range(len(eval_data)):
   audio_sample = eval_data[i]
   if os.path.exists(audio_sample["file"]):
     audio, _ = librosa.load(audio_sample["file"], sr=16000)
     test_str=audio_sample["text"].lower()
     stk=""
     for ele in test_str:
        if ele not in punc:
            stk+=ele
     ref.append(stk)

     input_values = processor(audio, return_tensors="pt",
                                     sampling_rate=sample_rate).input_values.to(device)
     with torch.no_grad():
        logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        log_processed = logits.cpu().numpy()[0]
     transcription = processor.batch_decode(predicted_ids)

     test_str=transcription[0].lower()
     stk=""
     for ele in test_str:
        if ele not in punc:
            stk+=ele
     res.append(stk)
     if i%1000 == 0:
        print(ref[i],res[i])

transformation = Compose([
    ToLowerCase(),
    RemoveWhiteSpace(replace_by_space=True),
    RemoveMultipleSpaces(),
    ReduceToListOfListOfWords(word_delimiter=" ")
])

out = process_words(ref,res)

print("WER Wav2Vec Model:", wer(ref,res,truth_transform=transformation,hypothesis_transform=transformation)*100, "Substitutions:", out.substitutions, " Deletions:", out.deletions, " Insertions:",out.insertions)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating test split: 0 examples [00:00, ? examples/s]

layne took many trips to africa after studying african history lan took many trips to africa after studying african history
WER Wav2Vec Model: 10.405257393209201 Substitutions: 141  Deletions: 12  Insertions: 37


In [ ]:
from transformers import Wav2Vec2ForCTC, AutoProcessor

model_id = "facebook/mms-1b-all"
target_lang = "eng"

processor = AutoProcessor.from_pretrained(model_id, target_lang=target_lang)
model = Wav2Vec2ForCTC.from_pretrained(model_id, target_lang=target_lang, ignore_mismatched_sizes=True).to(device)

wav_path="/scratch/project_2007260/cv_aged_voice/FinalDataset/ASR/wavp_eighties_combined_test"
text_path="/scratch/project_2007260/cv_aged_voice/FinalDataset/ASR/tid_eighties_combined_test"

wav_l = [temp.strip() for temp in open(wav_path).readlines()]
text_l = [temp.strip() for temp in open(text_path).readlines()]
df = pandas.DataFrame(data={"file": wav_l, "audio": wav_l, "text": text_l})

df.to_csv("test.csv", sep=",",index=False)

dataFiles = {'test': "test.csv"}
ld = load_dataset('csv', data_files=dataFiles)

eval_data = ld["test"]


sample_rate=16000
device="cuda:0"

ref = []
res = []
punc = '''!()-[]{};.:'"\,<>./?@#$%^&*_~'''

for i in range(len(eval_data)):
   audio_sample = eval_data[i]
   if os.path.exists(audio_sample["file"]):
     audio, sampling_rate= librosa.load(audio_sample["file"], sr=16000)
     test_str=audio_sample["text"].lower()
     stk=""
     for ele in test_str:
        if ele not in punc:
            stk+=ele
     ref.append(stk)

     inputs = processor(audio, sampling_rate=sampling_rate, return_tensors="pt").to(device)
     with torch.no_grad():
        logits = model(**inputs).logits
     predicted_ids = torch.argmax(logits, dim=-1)
     transcription = processor.batch_decode(predicted_ids)

     test_str=transcription[0].lower()
     stk=""
     for ele in test_str:
        if ele not in punc:
            stk+=ele
     res.append(stk)
     if i%1000 == 0:
        print(ref[i],res[i])

transformation = Compose([
    ToLowerCase(),
    RemoveWhiteSpace(replace_by_space=True),
    RemoveMultipleSpaces(),
    ReduceToListOfListOfWords(word_delimiter=" ")
])

out = process_words(ref,res)

print("WER from  Model MMS:", wer(ref,res,truth_transform=transformation,hypothesis_transform=transformation)*100, "Substitutions:", out.substitutions, " Deletions:", out.deletions, " Insertions:",out.insertions)


Generating test split: 0 examples [00:00, ? examples/s]

layne took many trips to africa after studying african history lan took many trips africa after studying african history
WER from  Model MMS: 10.952902519167578 Substitutions: 156  Deletions: 14  Insertions: 30


In [ ]:
processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft").to("cuda:0")
wav_path="/scratch/project_2007260/cv_aged_voice/FinalDataset/ASR/wavp_eighties_combined_test"
text_path="/scratch/project_2007260/cv_aged_voice/FinalDataset/ASR/tid_eighties_combined_test"

wav_l = [temp.strip() for temp in open(wav_path).readlines()]
text_l = [temp.strip() for temp in open(text_path).readlines()]
df = pandas.DataFrame(data={"file": wav_l, "audio": wav_l, "text": text_l})

df.to_csv("test.csv", sep=",",index=False)

dataFiles = {'test': "test.csv"}
ld = load_dataset('csv', data_files=dataFiles)

eval_data = ld["test"]


sample_rate=16000
device="cuda:0"

ref = []
res = []
punc = '''!()-[]{};.:'"\,<>./?@#$%^&*_~'''

for i in range(len(eval_data)):
    audio_sample = eval_data[i]
    if os.path.exists(audio_sample["file"]):
        audio, sampling_rate= librosa.load(audio_sample["file"], sr=16000)
        test_str=audio_sample["text"].lower()
        stk=""
        for ele in test_str:
            if ele not in punc:
                stk+=ele
        ref.append(stk)

        inputs = processor(audio, sampling_rate=sampling_rate, return_tensors="pt").to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
            predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)

        test_str=transcription[0].lower()
        stk=""
        for ele in test_str:
            if ele not in punc:
                stk+=ele
        res.append(stk)
        if i%1000 == 0:
            print(ref[i],res[i])

transformation = Compose([
    ToLowerCase(),
    RemoveWhiteSpace(replace_by_space=True),
    RemoveMultipleSpaces(),
    ReduceToListOfListOfWords(word_delimiter=" ")
])

out = process_words(ref,res)

print("WER from Hubert Model:", wer(ref,res,truth_transform=transformation,hypothesis_transform=transformation)*100, "Substitutions:", out.substitutions, " Deletions:", out.deletions, " Insertions:",out.insertions)


Generating test split: 0 examples [00:00, ? examples/s]

layne took many trips to africa after studying african history lan took many trips to africa after studying african history
WER from Hubert Model: 11.610076670317634 Substitutions: 155  Deletions: 16  Insertions: 41
